# Introduction to Clustering


<p><div style="float:right;margin-left:5px;"><img src="https://cambridge-intelligence.com/wp-content/uploads/2016/01/clustering-animated.gif" width="500"/></div></p>
This notebook is short and sweet and covers both hierarchical clustering and k-means clustering.

Let's start with the example we did in class, the dog breed dataset.

First, we will load the dataset:

In [3]:
import pandas as pd
dog_data = pd.read_csv('https://raw.githubusercontent.com/zacharski/machine-learning/master/data/dogbreeds.csv')
dog_data = dog_data.set_index('breed')

In [4]:
dog_data

,height (inches),weight (pounds)
breed,,
Border Collie,20,45
Boston Terrier,16,20
Brittany Spaniel,18,35
Bullmastiff,27,120
Chihuahua,8,8
German Shepherd,25,78
Golden Retreiver,23,70
Great Dane,32,160
Portuguese Water Dog,21,50


Looking at the values in the height and weight columns it looks like we should normalize the data.

<img src="http://animalfair.com/wp-content/uploads/2014/08/chihuahua-and-great-dane.jpg" width="700"/>


In [6]:
## TODO


And let's visualize that data:

In [8]:
from bokeh.charts import Scatter, output_file, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
x = Scatter(dog_data, x='weight (pounds)', y='height (inches)', title="Plot of Dog Breeds",
            xlabel="Normalized Weight", ylabel="Normalized Height")

output_file("dogbreed.html")

show(x)

Loading BokehJS ...

Gazing at the scatter plot, it looks like we could group the data into three clusters. There are the 2 data points on the bottom left (*Chihuahua* and *Yorkshire Terrier*) The top right group of two (*Bull Mastiff* and *Great Dane*) and the middle group with all the other breeds. 


## k means clustering

Instead of covering what is in the textbook you may want to scope out the totally awesome [clustering chapter](http://guidetodatamining.com/assets/guideChapters/DataMining-ch8.pdf) (I am joking about the *totally awesome* bit)

Some tiny amount of extra xp for knowing trivia.

1. there are 2 pictures of people on page 8.5 and a picture of a person on 8.6. What do these people have in common? This requires world knowledge (like who the people are). Do not share your knowledge with other. (you can give exactly one person an obscure hint)
2. There are also people on 8.29 and 8.30. What do these people have in common? The same restrictions apply.


This was just a ruse to get you to look at--and possibly read--the chapter which as you know is totally awesome.

Okay, back to real work. Let's divide our dog dataset into 3 clusters:


In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, ).fit(dog_data)
labels = kmeans.labels_

The variable `labels` is an array the specifies which group each dog belongs to:

In [10]:
labels

array([0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2], dtype=int32)

My results were:

    array([0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2], dtype=int32)

which indicates that the first, second, and third dogs are in group 0, the next one in group 1 and so on  That may be helpful for future computational tasks but is not the helpful if we are trying to visualize the data. Let me munge that a bit into a slightly more useful form:

In [11]:
groups = {0: [], 1: [], 2: []}
i = 0
for index, row in dog_data.iterrows():
    groups[labels[i]].append(index)
    i += 1
## Now I will print it in a nice way:

for key, value in groups.items():
    print ('CLUSTER %i' % key)
    for breed in value:
        print("    %s" % breed)
    print('\n')

CLUSTER 0
    Border Collie
    Boston Terrier
    Brittany Spaniel
    German Shepherd
    Golden Retreiver
    Portuguese Water Dog
    Standard Poodle


CLUSTER 1
    Bullmastiff
    Great Dane


CLUSTER 2
    Chihuahua
    Yorkshire Terrier




keep in mind that since they initial centroids are selected somewhat randomly it is possible that you get a different answer than I do. The answer I got was:

    CLUSTER 0
        Border Collie
        Boston Terrier
        Brittany Spaniel
        German Shepherd
        Golden Retreiver
        Portuguese Water Dog
        Standard Poodle
    

    CLUSTER 1
        Bullmastiff
        Great Dane


    CLUSTER 2
        Chihuahua
        Yorkshire Terrier


## Hierarchical Clustering

For the basics on hierarchical clustering consult [the fine chapter I mentioned](http://guidetodatamining.com/assets/guideChapters/DataMining-ch8.pdf). Here is how to do hierarchical clustering in sklearn.

In [14]:

from sklearn.cluster import AgglomerativeClustering
clusterer = AgglomerativeClustering(affinity='euclidean', linkage='ward')
clusterer.fit(dog_data)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=2,
            pooling_func=<function mean at 0x115798d90>)

we can get the highest level division by viewing the `.labels_`:


In [18]:
clusterer.labels_

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

So here the first dog breed, Border Collie belongs to cluster 0. Keep in mind that in kmeans there is a random element--the selection of the initial centroids, but in hierarchical clustering there is no randomness so you should get the exact same answer I do. so that is the high level division but the hierarchical clustering algorithm constructs a tree - specifically, a dendrogram. To view that requires some imagination.  I can print a representation of the tree by:

In [19]:
import itertools
ii = itertools.count(dog_data.shape[0])
[{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in clusterer.children_]

[{'left': 0, 'node_id': 11, 'right': 8},
 {'left': 4, 'node_id': 12, 'right': 10},
 {'left': 5, 'node_id': 13, 'right': 6},
 {'left': 1, 'node_id': 14, 'right': 2},
 {'left': 9, 'node_id': 15, 'right': 11},
 {'left': 14, 'node_id': 16, 'right': 15},
 {'left': 3, 'node_id': 17, 'right': 7},
 {'left': 13, 'node_id': 18, 'right': 16},
 {'left': 12, 'node_id': 19, 'right': 18},
 {'left': 17, 'node_id': 20, 'right': 19}]

The first line `{'left': 0, 'node_id': 11, 'right': 8}` reads that we combine cluster 0 *Border Collie* with cluster 8 *Portuguese Water Dog* to create Cluster 11. The next line says we  combine 4 *Chihuahua* with 10 *Yorkshire Terrier* to create cluster 12.  Let's associate index numbers with the dog breed names so that structure is easier to parse:


In [14]:
dog_names = pd.DataFrame({'breeds': dog_data.index.values})
dog_names

,breeds
0,Border Collie
1,Boston Terrier
2,Brittany Spaniel
3,Bullmastiff
4,Chihuahua
5,German Shepherd
6,Golden Retreiver
7,Great Dane
8,Portuguese Water Dog
9,Standard Poodle


That makes it easier to interpret lines like:

    {'left': 1, 'node_id': 14, 'right': 2},
    
We are combining `1` *Boston Terrier* and `2` *Brittany Spaniel*

So when we draw this out we get:

<img src="http://zacharski.org/files/courses/cs419/dendro.png" width="700"/>


<h1 style="color:red">Tasks</h1>

<h2 style="color:red">Task 1: Breakfast Cereals</h2>
I would like you to create 4 clusters of the data in:

    https://raw.githubusercontent.com/zacharski/pg2dm-python/master/data/ch8/cereal.csv
    
For clustering use the features calories, sugar, protein, and fiber.

Print out the results as we did for the dog breed data:


    CLUSTER 0
    Bullmastiff
    Great Dane
    

    CLUSTER 1
        Chihuahua
        Yorkshire Terrier
    

    CLUSTER 2
        Border Collie
        Boston Terrier
        Brittany Spaniel
        German Shepherd
        Golden Retreiver
        Portuguese Water Dog
        Standard Poodle
        
Because the initial centroids are random, by default the sklearn kmeans agorithm runs the algorithm 10 times and picks the best results (based on some of squares error). I would like you to change that parameter so it runs the algorithm 100 times.   Just google `sklearn kmeans` to get documentation on the parameters.


<h2 style="color:red">Task 2: Hierarchical</h2>
I would like you to use the hierarchical clustering algorithm on the cereal data.


And here is a question. What clusters with *Fruity Pebbles*?

I am keeping this notebook short just to keep it more doable. Shortly I will post a lab asking you to cluster a slightly larger dataset.